<a href="https://colab.research.google.com/github/ArtyA23/Maker_Portfolio_Colab_Code/blob/main/Preprocessing_code_emotional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Here, I have a dataset I used that I considered very important - my emotional dataset. This attempts to give my AI some therapeutic knowledge, not to replace therapy, but to mimic it at a small scale. This really gives my AI that human touch, making it more understanding than other models.

In [8]:
import re
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import Dataset

raw = load_dataset("thu-coai/esconv")

print(dataset['train'][0])

{'text': '{"experience_type": "Current Experience", "emotion_type": "anxiety", "problem_type": "job crisis", "situation": "I am on short term disability and I am afraid I will lose my job if I don\'t go back soon.", "survey_score": {"seeker": {"initial_emotion_intensity": "3", "empathy": "5", "relevance": "5", "final_emotion_intensity": "2"}, "supporter": {"relevance": "5"}}, "dialog": [{"text": "Hello good afternoon.", "speaker": "usr"}, {"text": "Hi, good afternoon.", "speaker": "sys", "strategy": "Question"}, {"text": "I\'m feeling anxious that I am going to lose my job.", "speaker": "usr"}, {"text": "Losing a job is always anxious.", "speaker": "sys", "strategy": "Reflection of feelings"}, {"text": "I hope I don\'t.", "speaker": "usr"}, {"text": "Why do you think you will lose your job?", "speaker": "sys", "strategy": "Question"}, {"text": "I am on short term disability and I am not ready to go back to work yet but I do not have any job protection.", "speaker": "usr"}, {"text": "Oh

In [12]:
def clean(text):
    text = re.sub(r"\s+", " ", text.strip())
    text = re.sub(r"http\S+", "", text)
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')
    return text

def convo_to_pairs(example, role_user_set=None, role_assistant_set=None):
    if role_user_set is None:
        role_user_set = {"seeker", "user", "client", "asker"}
    if role_assistant_set is None:
        role_assistant_set = {"supporter", "helper", "responder"}

    msgs = example.get("text") or example.get("messages") or []

    turns = []
    for m in msgs:
        if isinstance(m, dict):
            role = (m.get("role") or m.get("speaker") or "").strip().lower() or "speaker"

            if role in ("helper", "responder", "supporter"):
                role = "supporter"
            if role in ("user", "seeker", "client", "asker"):
                role = "seeker"
            utter = clean(m.get("text") or m.get("utterance") or m.get("content") or "")
            turns.append(f"{role}: {utter}")
        elif isinstance(m, str):
            turns.append(clean(m))
        else:
            turns.append(clean(str(m)))

    pairs = []

    for i in range(1, len(turns)):
        prev_role = turns[i-1].split(":", 1)[0].strip().lower()
        curr_role = turns[i].split(":", 1)[0].strip().lower()
        if prev_role in role_user_set and curr_role in role_assistant_set:
            context = " \n".join(turns[:i])
            response = turns[i].split(":", 1)[1].strip()
            pairs.append({"context": context, "response": response})
    return {"pairs": pairs}

raw = raw.map(convo_to_pairs)

pair_rows = []
for split in raw.keys():
    for ex in raw[split]:
        for p in ex.get("pairs", []):
            pair_rows.append(p)

pairs_ds = Dataset.from_list(pair_rows)

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

pad_id = tokenizer.pad_token_id

def tokenize_and_mask(batch):
    input_ids_batch = []
    attention_masks = []
    labels_batch = []

    for ctx, resp in zip(batch["context"], batch["response"]):
        ctx_ids = tokenizer(ctx, add_special_tokens=False)["input_ids"]
        resp_ids = tokenizer(resp, add_special_tokens=False)["input_ids"]

        input_ids = ctx_ids + resp_ids
        labels = [-100] * len(ctx_ids) + resp_ids

        if len(input_ids) > MAX_LENGTH:
            trim = len(input_ids) - MAX_LENGTH
            input_ids = input_ids[trim:]
            labels = labels[trim:]

        pad_len = MAX_LENGTH - len(input_ids)
        if pad_len > 0:
            input_ids = input_ids + [pad_id] * pad_len
            labels = labels + [-100] * pad_len

        attn = [1 if tid != pad_id else 0 for tid in input_ids]

        input_ids_batch.append(input_ids)
        attention_masks.append(attn)
        labels_batch.append(labels)

    return {"input_ids": input_ids_batch, "attention_mask": attention_masks, "labels": labels_batch}

tokenized_pairs = pairs_ds.map(tokenize_and_mask, batched=True, remove_columns=pairs_ds.column_names)

example = tokenized_pairs[0]
print("input_ids len:", len(example["input_ids"]))
print("labels sample (first 50):", example["labels"][:50])
first_label_idx = next((i for i, v in enumerate(example["labels"]) if v != -100), None)
print("first non-masked token index:", first_label_idx)
print("decoded tail (response tokens):", tokenizer.decode(example["input_ids"][first_label_idx:]).strip())

tokenized_pairs.save_to_disk("processed_data-emotional")


Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

IndexError: Invalid key: 0 is out of bounds for size 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
da.save_to_disk("/content/drive/MyDrive/Attreya_AI_Colab_Project/Attreya_Maker_Portfolio-rasA/Data-ONLY_FOR_VIEWING/processed_data-emotional")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving the dataset (0/1 shards):   0%|          | 0/910 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/195 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/195 [00:00<?, ? examples/s]

In [ ]:
import os
model_path = "/content/drive/MyDrive/Attreya_AI_Colab_Project/Attreya_Maker_Portfolio-rasA/Data-ONLY_FOR_VIEWING/processed_data-emotional"
print(os.path.exists(model_path))
print(os.listdir(model_path))

True
['dataset_dict.json', 'train', 'validation', 'test']
